* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [1]:
import importlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [2]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [3]:
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [4]:
## y is binary.
display(train["y"].describe())

count    592380.000000
mean          0.356958
std           0.479103
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: y, dtype: float64

In [5]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [6]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [7]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

# Split back into train and test, and build model

In [8]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [9]:
train_no_nan = train.fillna(-100)
test_no_nan = test.fillna(-100)

In [10]:
train_no_nan.isna()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid_vol_cols_min,bid_vol_cols_spread,ask_cols_max,ask_cols_min,ask_cols_spread,ask_vol_cols_max,ask_vol_cols_min,ask_vol_cols_spread,last_price_div__mid,date
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592376,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592377,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
592378,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
# train.drop(["id"],axis=1).to_csv("train_hft.csv.gz",index=False,compression="gzip")
# test.to_csv("test_hft_nodates.csv.gz",index=False,compression="gzip")

In [12]:
# we don't know if the test set has a temporal split, so we'll just try a random split for now
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [13]:
print(X.shape)
print(np.isfinite(X).all())

(592380, 39)
last_price              True
mid                     True
opened_position_qty     True
closed_position_qty     True
transacted_qty          True
d_open_interest         True
bid1                    True
bid2                    True
bid3                    True
bid4                    True
bid5                    True
ask1                    True
ask2                    True
ask3                    True
ask4                    True
ask5                    True
bid1vol                 True
bid2vol                 True
bid3vol                 True
bid4vol                 True
bid5vol                 True
ask1vol                 True
ask2vol                 True
ask3vol                 True
ask4vol                 True
ask5vol                 True
bid_cols_max            True
bid_cols_min            True
bid_cols_spread         True
bid_vol_cols_max        True
bid_vol_cols_min        True
bid_vol_cols_spread     True
ask_cols_max            True
ask_cols_min            True
a

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

grad_tree = AdaBoostClassifier()

n_estimators = range(10,201,10)
learning_rate = 10.0**np.arange(-1,-3,-1)

param_grid = dict(n_estimators=n_estimators,learning_rate=learning_rate)


In [15]:
random = RandomizedSearchCV(estimator=grad_tree,
                            param_distributions=param_grid,
                            scoring='roc_auc',
                            verbose=10, n_jobs=3,
                            n_iter=50)

In [16]:
random_result = random.fit(X, y)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   15.5s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:  2.1min
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  5.6min
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  8.9min
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed: 15.0min
[Parallel(n_jo

In [17]:
print(random_result.best_score_)

0.6434327084571398


In [18]:
random_result.predict_proba(test_no_nan.drop(["id","date","y"],axis=1))

array([[0.50094622, 0.49905378],
       [0.55658536, 0.44341464],
       [0.55806232, 0.44193768],
       ...,
       [0.51589126, 0.48410874],
       [0.5394718 , 0.4605282 ],
       [0.51751961, 0.48248039]])

In [19]:
print(random_result.get_params())

{'cv': 'warn', 'error_score': 'raise-deprecating', 'estimator__algorithm': 'SAMME.R', 'estimator__base_estimator': None, 'estimator__learning_rate': 1.0, 'estimator__n_estimators': 50, 'estimator__random_state': None, 'estimator': AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None), 'fit_params': None, 'iid': 'warn', 'n_iter': 50, 'n_jobs': 3, 'param_distributions': {'n_estimators': range(10, 201, 10), 'learning_rate': array([0.1 , 0.01])}, 'pre_dispatch': '2*n_jobs', 'random_state': None, 'refit': True, 'return_train_score': 'warn', 'scoring': 'roc_auc', 'verbose': 10}


## export predictions

In [20]:
# test["Predicted"] = model.predict(test.drop(["id","date","y"],axis=1),prediction_type='Probability')[:,1]
test["Predicted"] = random_result.predict_proba(test_no_nan.drop(["id","date","y"],axis=1))[:,1]
test[["id","Predicted"]].to_csv("submission_test_ada_boost3.csv",index=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
